In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    AutoConfig,
)
from pytorch_lightning import LightningModule
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./out-small-1.1')

In [ ]:
tokenizer.push_to_hub('nanot5-small-malaysian-cased', organization='mesolitica')

In [ ]:
class Module(LightningModule):
    def __init__(self):
        super().__init__()
        config = AutoConfig.from_pretrained(
            './out-small-1.1'
        )
        self.model = T5ForConditionalGeneration.from_pretrained(
            './out-small-1.1',
            config=config,
        )

In [ ]:
!ls logs/small

In [ ]:
model = Module()

In [ ]:
weights = model.state_dict()

In [ ]:
old_weights = torch.load('logs/small/model-epoch=02-step=62000.ckpt',
                             map_location=torch.device('cpu'))['state_dict'].items()

In [ ]:
for k, v in old_weights:
    new_k = k.replace('._orig_mod', '')
    print(k, new_k)
    weights[new_k] = v

In [ ]:
model.load_state_dict(weights)

In [ ]:
model.model.push_to_hub('nanot5-small-malaysian-cased', organization='mesolitica')